In [91]:
from stocksymbol import StockSymbol
import pandas as pd

ss = StockSymbol("fb6844dd-04c4-4a1e-9596-6b451a0461b3")
us_stock = pd.DataFrame(ss.get_symbol_list(market="US", index=None))

In [86]:
from difflib import SequenceMatcher

def sequence_similarity(str1, str2):
    # print(str1, str2)
    matcher = SequenceMatcher(None, str1, str2)
    return matcher.ratio()

In [2]:
from asset_module import Asset

asset = Asset()

asset.find("apple")

0


symbol             AAPL
shortName         apple
longName     Apple Inc.
exchange         NASDAQ
market        us_market
quoteType        EQUITY
Name: 0, dtype: object

In [82]:
def find_asset(search_name):
    distances = []
    for asset_name in us_stock["shortName"]:
        # print(asset_name)
        distances.append(sequence_similarity(asset_name.lower(), search_name.lower()))

    # print(distances)
    highest_similarity = distances.index(max(distances))
    print(highest_similarity)
    return us_stock.iloc[highest_similarity, :]

In [84]:
find_asset("micr")

6556


symbol             MICT
shortName          mict
longName     MICT, Inc.
exchange         NASDAQ
market        us_market
quoteType        EQUITY
Name: 6556, dtype: object

,symbol,shortName,longName,exchange,market,quoteType
0,AAPL,apple,Apple Inc.,NASDAQ,us_market,EQUITY
1,MSFT,microsoft,Microsoft Corporation,NASDAQ,us_market,EQUITY
2,GOOG,alphabet,Alphabet Inc.,NASDAQ,us_market,EQUITY
3,GOOGL,alphabet,Alphabet Inc.,NASDAQ,us_market,EQUITY
4,AMZN,amazon,"Amazon.com, Inc.",NASDAQ,us_market,EQUITY
...,...,...,...,...,...,...
8775,GNAL,,"Generation Alpha, Inc.",OTC,us_market,EQUITY
8776,ZIMCF,,ZIM Corp.,OTC,us_market,EQUITY
8777,FLCX,,flooidCX Corp.,OTC,us_market,EQUITY
8778,BLEG,,BRANDED LEGACY INC.,OTC,us_market,EQUITY
